```{contents}
:depth: 4
```

# Gradiente conjugado

## Gradiente conjugado

### Métodos de dirección de descenso

Supongamos que queremos minimizar funciones cuadráticas convexas $f:\mathbb{R}^n→\mathbb{R}$ de la forma 

$$ f(x)=\frac{1}{2}x^tAx−x^tb $$ 


donde $x,b\in \mathbb{R}^n$ y $A\in \mathbb{R}^{n\times n} $ es una matriz simétrica positiva definida.


Para minimizar $f (x)$ calculamos primero su gradiente 

$$\nabla f(x)=\nabla \left(\frac{1}{2}x^tAx−x^tb\right)=Ax-b, $$

igualando a cero:

$$A x=b,$$

es decir, para minimizar $f$ debemos resolver un sistema lineal de ecuaciones.


Si $A$ es simétrica y positiva definida, entonces podemos definir un producto interior:

$$\langle u,v \rangle_A:= u^t A v=\langle A^t u,v \rangle  = \langle Au,v  \rangle = \langle u,A v \rangle   $$

### Método del  descenso del gradiente 

Una opción intuitiva para elegir la dirección de descenso $d$ es la dirección opuesta al gradiente $\nabla f(x)$, de ahí el nombre del método. 

$d=-\nabla f(x)=-(Ax-b)=b-Ax$

de modo que buscamos el mínimo de $f$ sobre la recta: $x+\alpha d$

esto es, encontrar un $\alpha$ que minimice $f$: 

$$\min_{\alpha}\ f(x+\alpha d)$$

$$\begin{align*}
f\left(x+\alpha d\right)&=\frac{1}{2}\left(x+\alpha d\right)^tA\left(x+\alpha d\right)−\left(x+\alpha d\right)^tb\\
&= \frac{1}{2}\left(x^t+\alpha d^t\right)\left(Ax+\alpha Ad\right)−\left(x^tb+\alpha d^tb\right)\\
&= \frac{1}{2}\left(x^t+\alpha d^t\right)Ax+\frac{1}{2}\alpha \left(x^t+\alpha d^t\right)Ad−\left(x^tb+\alpha d^tb\right)\\
&= \frac{1}{2}(x^tAx+\alpha d^tAx)+\frac{1}{2} \left(\alpha x^tAd+\alpha^2 d^tAd \right) −\left(x^tb+\alpha d^tb\right)\\
&= \frac{1}{2}\alpha^2 d^tAd+\alpha \left( \frac{1}{2}d^tAx+ \frac{1}{2}x^tAd-d^tb\right)+\left(\frac{1}{2}x^tAx-x^tb\right)\\
&= \frac{1}{2}\alpha^2 d^tAd+\alpha \left( d^tAx-d^tb\right)+\left(\frac{1}{2}x^tAx-x^tb\right)\\
&= \frac{1}{2}\alpha^2 d^tAd+\alpha d^t\left( Ax-b\right)+\left(\frac{1}{2}x^tAx-x^tb\right)
\end{align*}
$$

Derivando respecto de $\alpha$ e igualando a cero obtenemos: 

$$\alpha=-\frac{d^t (Ax-b)}{d^tAd}$$


El método de descenso de gradiente es:

* Entrada: $A, x_0, b, \varepsilon$

    $i \leftarrow 0$
    
* Repetir  
    $d_i \leftarrow b-Ax_{i}$

    $\alpha_i \leftarrow \frac{d_i^td_i}{d_i^t A d_i} $

    $x_{i+1} \leftarrow x_i+\alpha_id_i$ 

    $i   \leftarrow i+1 $
 
 Hasta que $\lVert d_i \rVert < \varepsilon$ 



In [ ]:
using PGFPlotsX, Contour, LaTeXStrings, Latexify, Printf, LinearAlgebra

In [ ]:
"""
Rutina para resolver el sistema de ecuaciones lineales `Ax=b` por método de descenso del gradiente

Entrada: 

* `A`  arreglo n×n 
* `b`  arreglo n×1 
* `itmax`  número máximo de iteraciones
* `tol`  tolerancia a la aproximación

Salida:

* `x` arreglo con la aproximación de la solución
* `X` conjunto de aproximaciones de x
"""
function miG(A::Array{Float64},b::Array{Float64}, itmax::Int64, tol::Float64)
        n = length(b)

        x = zeros(n,1)    # aprox inicial en el origen
        it = 0        # contador 
        nd = norm(b)        # tamaño del residual
        XX=[]
        # CICLO PRINCIPAL
            while nd ≥ tol # criterio de paro por tamaño del residual
                XX=push!(XX,x)
        
                # muestra tamaño del gradiente
                cad = @sprintf "%1.5e" nd
                display(latexstring("\$\\|d_{$it}\\|\$ = "*cad))
                d = b-A*x       # nuevo gradiente ortogonal al anterior
                α  = d'*d / (d'*A*d)                # mínimo sobre la línea
                x +=  α[1]*d               # actualiza la aproximación
                it += 1                # actualiza contador
                ( it ≥ itmax ) && break                # criterio de paro por iteraciones
                nd=norm(d)
            end
    
            if ( it < itmax ) 
               println("salí por tamaño de gradiente")
            else
               println("salí por iteraciones") 
            end

            cad = @sprintf "%1.5e" nd            # muestra tamaño del último residual
            display(latexstring("\$\\|g_{$it}\\|\$ = "*cad))
            XX=push!(XX,x)
            XX=[( i[1],i[2] ) for i in XX]

    return x,XX
end

In [ ]:
A=[6.0 -5.0; -5.0 5.0]
b=[4.0;0.0]
A\b

In [ ]:
latexify(A)

In [ ]:
x,XX=miG(A,b, 1000,  1e-5);

In [ ]:
X = LinRange((x[1]-5),(x[1]+5),50)
Y = LinRange((x[2]-5),(x[2]+5),50)
R=[ (i,j) for  i in X, j in Y];

In [ ]:
f = (u) -> ((1/2)*[u[1] u[2]] *A*[u[1]; u[2]])[1]-b'*[u[1]; u[2]]
c =  contours(X, Y, f.(R), 20)

In [ ]:
    plt = @pgf Axis(
    {
        xlabel = "x",
        ylabel = "y",
    },
     Plot({
     color = "red",
    },
    Table(c)),
    Plot(
        {
        },
        Coordinates(XX)
    )
)

In [ ]:
display(plt)

### Método del  gradiente conjugado

**Definición:** Decimos que dos vectores $u,v$ no nulos, son conjugados con respecto a $A$ si:

$$\langle u,v \rangle_A=0, \text{ con } u\neq v$$

es decir, dos vectores son conjugados si son ortogonales con respecto a este producto interior

**Observación:** Supongamos que $\lbrace p_1, p_2,\ldots ,p_n\rbrace$ es un conjunto de direcciones mutuamente conjugadas, es decir, que cumplen 

$$p_i^tAp_j=0 \text{ para } i\neq j$$

El conjunto $\lbrace p_1, p_2,\ldots ,p_n\rbrace$ es una base de $\mathbb{R}^n$. 

Esto es, si $x^*$ es la solución de $Ax = b$ $$Ax^*=b$$ entonces: 

$$x^*=\alpha_1 p_1+\alpha_2 p_2+\ldots+\alpha_n p_n,$$

los coeficientes están dados a partir de la combinación lineal

$$A x^*=\alpha_1 A p_1+\alpha_2 A p_2+\ldots+\alpha_n A p_n=b$$

multiplicando por $p_i^t$

$$p_i^tA x^*=p_i^t(\alpha_1 A p_1+\alpha_2 A p_2+\ldots+\alpha_n A p_n)=p_i^tb$$

$$\alpha_ip_i^t A p_i =p_i^tb$$

$$\therefore \alpha_i =\frac{p_i^tb}{p_i^t A p_i}=\frac{\langle p_i,b  \rangle}{\langle p_i,p_i  \rangle}_A=\frac{\langle p_i,b  \rangle}{\lVert p_i \rVert^2_A }  $$

### Idea del método

* La idea del algoritmo es utilizar direcciones conjugadas para el descenso en la búsqueda del punto
óptimo $x^*$, es decir:

* Primero encontramos una secuencia de $n$ direcciones conjugadas y luego computamos los coeficientes $\alpha_i$.

* El algoritmo de gradiente conjugado garantiza la obtención de una solución en un máximo de $n$ iteraciones.


El algoritmo es el siguiente:

* Entrada: $A, x_0, b, \varepsilon$

    $r_0 \leftarrow Ax_0-b$

    $p_0 \leftarrow -r_0$

    $i \leftarrow 0$
    
* Mientras mientras $\lVert r_i \rVert > \varepsilon$  

    $\alpha_i \leftarrow -\frac{r_i^tp_i}{p_i^t A p_i} $

    $x_{i+1} \leftarrow x_i+\alpha_ip_i$ 

    $r_{i+1} \leftarrow Ax_{i+1}-b$

    $\beta_{i+1}   \leftarrow \frac{r_{i+1}^tr_{i+1}}{r_i^t r_i} $

    $p_{i+1}   \leftarrow -r_{i+1}+\beta_{i+1}p_i$
    
    $i   \leftarrow i+1 $

Generalmente no es necesario realizar las $n$ iteraciones, se puede definir la precisión deseada limitando la convergencia con una tolerancia $\varepsilon$.

El proceso más lento del algoritmo es la multiplicación matriz-vector

In [ ]:
"""
Rutina para resolver el sistema de ecuaciones lineales `Ax=b` por método del gradiente conjugado

Entrada: 

* `A`  arreglo n×n 
* `b`  arreglo n×1 
* `itmax`  número máximo de iteraciones
* `tol`  tolerancia a la aproximación

Salida:

* `x` arreglo con la aproximación de la solución
"""
function miGC(A::Array{Float64},b::Array{Float64}, itmax::Int64, tol::Float64)
        n = length(b)

        x = vec(zeros(n))    # aprox inicial en el origen
        r = -vec(b)        # gradiente inicial = residual inicial = - lado derecho 
        p = -r       # primera dirección contraria al gradiente inicial
        it = 0        # contador 
        nr = norm(r)        # tamaño del residual
        XX=[]

        # CICLO PRINCIPAL
            while nr ≥ tol # criterio de paro por tamaño del gradiente
                XX=push!(XX,x)

                # muestra tamaño del residual
                cad = @sprintf "%1.5e" nr
                display(latexstring("\$\\|r_{$it}\\|\$ = "*cad))
        
                α  = -r'*p / (p'*A*p)                # mínimo sobre la línea
                x +=  α[1]*p                # actualiza la aproximación
                rnew = A*x-b        # nuevo residual ortogonal al anterior
                β = ((rnew'*rnew) / (r'*r))[1]        # coeficientes para A-ortogonalidad de las direcciones
                r = rnew                # actualiza gradiente
                nr = norm(r)                # tamaño del residual
                p = -r + β*p                # actualiza dirección
                it += 1                # actualiza contador
                ( it ≥ itmax ) && break                # criterio de paro por iteraciones
            end
    
            if ( it < itmax ) 
               println("salí por tamaño de gradiente")
            else
               println("salí por iteraciones") 
            end

            cad = @sprintf "%1.5e" nr            # muestra tamaño del último residual
            display(latexstring("\$\\|r_{$it}\\|\$ = "*cad))
                XX=push!(XX,x)

                XX=[( i[1],i[2] ) for i in XX]

    return x,XX
end

x,XXc=miGC(A,b, 300,  1e-5);

In [ ]:
X = LinRange((x[1]-5),(x[1]+5),50)
Y = LinRange((x[2]-5),(x[2]+5),50)
R=[ (i,j) for  i in X, j in Y];

In [ ]:
f = (u) -> ((1/2)*[u[1] u[2]] *A*[u[1]; u[2]])[1]-b'*[u[1]; u[2]]
c =  contours(X, Y, f.(R), 20)

In [ ]:
    plt = @pgf Axis(
    {
        xlabel = "x",
        ylabel = "y",
    },
     Plot({
     color = "red",
    },
    Table(c)),
    Plot(
        {
        },
        Coordinates(XXc)
    ),
    
    Plot(
        {
        },
        Coordinates(XX)
    )
    
)

In [ ]:
display(plt)

### Paquete IterativeSolvers

In [ ]:
using IterativeSolvers

In [ ]:
?cg!

In [ ]:
n = length(b)
x = vec(zeros(n)) 

In [ ]:
cg!(x,A,b)

## Ecuación de Poisson 
$${\displaystyle \Delta \varphi =f\,}$$

### Ecuación de Poisson 1D

Dada la función $f:[0,1]\to\mathbb R$, 

dados $\alpha,\beta\in\mathbb R$,

hallar función $u:[0,1]\to \mathbb R$ tal que

> $$\begin{array}{rlc}
u''(x) & = -f(x), &  0 < x<1 \\ 
u(0) & =\alpha & \\
u(1) & =\beta & \\
\end{array}$$

Particionamos el dominio en $n$-subintervalos introduciendo los puntos $x_i = ih$ con $h=\frac{1}{n}$. 

De esta manera, para cada uno de los puntos interiores reemplazamos las derivadas por su aproximación de segundo orden en diferencias finitas (a partir de Taylor alrededor de $x_i$):

$$\frac{\partial^2}{\partial x^2} u(x_i) = \frac{u_{i-1} -2 u_i + u_{i+1}}{h^2} - \frac{h^2}{12} \frac{\partial^4}{\partial x^4}u(\xi_i)$$

con $\xi_i \in (x_{i-1}, x_{i+1})$ por lo que:

$$\frac{v_{i-1}-2v_i+v_{i+1}}{h^2} = -f_i$$

con un error de truncamiento local del orden de $O(h^2)$ y donde utilizamos la $v$ para hablar de la aproximación a la $u$. Si lo escribimos de manera matricial, tenemos:

$$\frac{1}{h^2} \begin{bmatrix} -2 & 1 & & & & & \\ 1 & -2 & 1 & & & & \\ & 1 & -2 & 1 & & & \\ & & \cdot & \cdot & \cdot & & \\ & & & \cdot & \cdot & \cdot & \\ & & & & 1 & -2 & 1 \\ & & & & & 1 & -2\end{bmatrix} \begin{bmatrix} v_1 \\ v_2 \\ v_3 \\ \cdot \\ \cdot \\ v_{n-2} \\ v_{n-1} \end{bmatrix} = - \begin{bmatrix} f_1 \\ f_2 \\ f_3 \\ \cdot \\ \cdot \\ f_{n-2} \\ f_{n-1} \end{bmatrix}$$

Existen otros esquemas de mayor orden para la aproximación, por ejemplo el esquema central de cuarto orden:

$$\frac{\partial^2}{\partial x^2} u(x_i) = \frac{-u_{i-2}+16u_{i-1}-30u_{i}+16u_{i+1}-u_{i+2}}{12 h^2} + \frac{h^4}{90} \frac{\partial^6}{\partial x^6}u(\xi_i)$$

con $\xi_i \in (x_{i-2},x_{i+2})$.

### Aproximación numérica de la solución

Dado $n\in\mathbb R$

Crea partición uniforme de $n+2$ puntos del intervalo cerrado $[0,1]$

quitale los extremos para obtener una partición $P: x_1,\dots,x_n$ de $n$ puntos del intervalo abierto $(0,1)$ 

con incremento $h=\frac{1}{n+1}$

Usando diferencias finitas centradas 

>$$\dfrac{-u(x_{i-1}) + 2u(x_{i}) -u(x_{i+1})  }{1/(n+1)^2} \approx f(x_i)$$

la ecuación de Poisson se puede escribir como el
sistema de ecuaciones

>$$Tu = b$$

donde 

>$$T=
\left[\begin{array}{rrrr}
 2 & -1 & & \\
 -1 & 2 & -1 & \\
  & \ddots & \ddots & \ddots  \\
  & & -1 & 2
\end{array}\right]_{n\times n}
\qquad
b = -\frac{1}{(n+1)^2}
\left[\begin{array}{r}
   f(x_1) \\  f(x_2)\\ \vdots \\ f(x_{n-1}) \\ f(x_n)
\end{array}\right]
+ 
\left[\begin{array}{r}
  \alpha \\  0 \\ \vdots \\ 0 \\ \beta
\end{array}\right]
$$



### Actividad

Crear rutina `prodT` para el producto de matriz $T$  por vector $u$

ENTRADA:
- arreglo 1D $u$ de $n$ componentes
SALIDA:
- arreglo 1D $y$ de $n$ componentes

realiza el producto $Tu$ sin formar la matriz $T$

Si $y=Tu$, ¿cuál es la componente $y_i$?

Modifica rutina `prodT` para que realize producto de matriz $T$ por matriz $U$

ENTRADA:
- arreglo 2D $U$ de $n\times n$
SALIDA:
- arreglo 2D $Y$ de $n\times n$ 

Si $Y=TU$, ¿como queda el renglón $i$ de$Y$?

In [ ]:
function prodT(X::Array{Float64})
    s  = size(X)
    TX = zeros(s)
    TX[1,:] = 2X[1,:] -X[2,:]    
    TX[2:end-1,:] = -X[1:end-2,:] + 2X[2:end-1,:] -X[3:end,:]
    TX[end,:] = -X[end-1,:] + 2X[end,:]
    return TX
end

Crear rutina `vecb` para formar el lado derecho $b$

ENTRADA:
- función $f$ (el lado derecho de la ecuación diferencial)
- entero $n$ (núm de puntos en partición de $(0,1)$)
- números flotantes $\alpha,\beta$ (condiciones de frontera)

SALIDA:
- arreglo 1D $b$ de $n$ componentes

In [ ]:
function vecb(funf::Function, px::Array{Float64}, u₀::Number,uₙ₊₁::Number)
    # núm de pts en partición
    n = length(px)
    # lado derecho de la ecuación diferencial en la partición
    fx = funf.(px)
    # lado derecho del sistema de ecuaciones lineales
    b  = fx/(n+1)^2
    b[1] += u₀
    b[end] += uₙ₊₁
    return b
end

Crear rutina `miCG` para resolver $Tu=b$ por el método de gradiente conjugado 

ENTRADA:
- función `miprod` (rutina para producto matriz vector)
- arreglo $b$ (el lado derecho del sistema de ecuaciones)
- entero *itmax*  (número máximo de iteraciones)
- número flotante *tol* (tolerancia para el tamaño del gradiente)

SALIDA:
- arreglo $u$ solución de $Tu=b$

En cada iteración muestra el tamaño del gradiente 
puede usar lo siguiente en su código

```julia

 cadena = @sprintf "%1.5e" normag
 display(latexstring("\$\\|g_{$it}\\|\$ = "*cadena))
```

In [ ]:
function miCG(fun::Function,b::Array{Float64}, itmax::Int64 = 300, tol::Float64 = 1e-5)
        #=
            Rutina para resolver sistema de ecuaciones lineales
                Ax=b
            por método del gradiente conjugado
    
            Entrada: 
                función fun para realizar producto matriz vector
                arreglo b   con el lado derecho
            Salida:
                arreglo x con la aproximación de la solución
        =#

            l = length(b)
            # aprox inicial en el origen
            x = zeros(l) 
            # gradiente inicial = residual inicial = - lado derecho
            g = -copy(b) 
            # primera dirección contraria al gradiente inicial
            d = -copy(g)
            # contador 
            it = 0
            # máximo numéro de iteraciones
            #itmax = 300
            # tamaño del gradiente
            ng = norm(g)
            # toleracia
            #tol = 1e-5
            # CICLO PRINCIPAL
            while ng ≥ tol # criterio de paro por tamaño del gradiente
                # muestra tamaño del gradiente
                cad = @sprintf "%1.5e" ng
                display(latexstring("\$\\|g_{$it}\\|\$ = "*cad))
                # rutina para producto matriz vector
                Ad = fun(d)
                # mínimo sobre la línea
                α  = (g'*g) / (d'*Ad)
                # actualiza la aproximación
                x +=  α*d
                # nuevo gradiente ortogonal al anterior
                gnew = g + α*Ad
                # coeficientes para A-ortogonalidad de las direcciones
                β = (gnew'*gnew) / (g'*g)
                # actualiza gradiente
                g = gnew
                # tamaño del gradiente
                ng = norm(g)
                # actualiza dirección
                d = -g + β*d
                # actualiza contador
                it += 1
                # criterio de paro por iteraciones
                ( it ≥ itmax ) && break
            end
            if ( it < itmax ) 
               println("salí por tamaño de gradiente")
            else
               println("salí por iteraciones") 
            end
            # muestra tamaño del último gradiente
            cad = @sprintf "%1.5e" ng
            display(latexstring("\$\\|g_{$it}\\|\$ = "*cad))
    return x
end

Crear rutina `Poisson1D` para resolver la Ec. de Poisson 1D por diferencias finitas

ENTRADA:
- función $f$ (el lado derecho de la ecuación diferencial)
- entero $n$ (núm de puntos en partición de $(0,1)$)
- números flotantes $\alpha,\beta$ (condiciones de frontera)

SALIDA:
- arreglo $\widehat u$ con los valores aproximados de la solución 
  $$ \widehat u = \left[\begin{array}{c}
   u(x_1) \\  u(x_2)\\ \vdots \\ u(x_{n-1}) \\ u(x_n)
\end{array}\right]$$

Esta rutina debe mandar a llamar a la rutina `miCG`

La rutina `miCG` para gradiente conjugado debe usar como entradas 

a la rutina `prodT` y el vector que devuelve  `ladob`

In [ ]:
function poisson1DCG(funf::Function, n::Int64, u₀::Number = 0,uₙ₊₁::Number = 0)
    # partición del intervalo [0,1] en n+2 pts
    px = LinRange(0,1,n+2)
    # partición del intervalo (0,1) en n pts
    px = collect(px)[2:end-1]
    # lado derecho del sistema de ecuaciones
    b = vecb(funf,px, u₀,uₙ₊₁)
    # solución del sistema de ecuaciones lineales por CG
    ûx = miCG(prodT,b)
    return px, ûx
end

<font color="blue">Pruebe su rutina `comparación1D` con la siguiente información</font>

In [ ]:
# lado derecho de ecuación diferencial
ff(x) =  cos(π*x)
# solución analítica a la ecuación de poisson con el lado derecho anterior 
u(x) = (cos(π*x) + 2.0x - 1.0)/π^2

In [ ]:
# partición y solución aprox en partición
px, ûx = poisson1DCG(ff,30)
# solución analítica en la partición
ux = u.(px);

In [ ]:
plt = @pgf Axis({
        height = "12cm",
        width  = "12cm",
        xlabel = L"$x$",
        ylabel = L"$u(x) = \frac{\pi}{2}(\cos(\pi x) + 2x - 1)$",
        title  = "solución de "*L"$-u''(x) = \cos(\pi x),\ u(0)=u(1)=0 $"*" por CG"
        },
        PlotInc({"no marks",  color="blue"},Coordinates(px,ux)),
        LegendEntry("exact"),
        PlotInc({"only marks",color="red"}, Coordinates(px,ûx)),
        LegendEntry("aprox")
    )

In [ ]:
# guarda gráfica de solución
pgfsave("sol1.pdf", plt; include_preamble = true, dpi = 150)